In [4]:
import os
import pandas as pd

In [5]:
file = "AL68663.screen.tab"

In [16]:
def extract_species(row):
    split_row = row['query-comment'].split(" ")
    if split_row[0].startswith("[") and split_row[1].endswith("]"):
        species = (" ").join([split_row[3], split_row[4]]) 
    else:
        species = (" ").join([split_row[1], split_row[2]])
    return species

In [20]:
def extract_accession(row):
    split_row = row['query-comment'].split(" ")
    if split_row[0].startswith("[") and split_row[1].endswith("]"):
        accession = split_row[2]
    else:
        accession = split_row[0]
    return accession

In [61]:
def import_mash_screen_to_pandas(screen_file):
    dataframe = pd.read_csv(file, sep="\t", names=['identity', 'shared-hashes',
                                                   'median-multiplicity', 'p-value',
                                                   'query-ID', 'query-comment'])
    
    dataframe['Species'] = dataframe.apply(extract_species, axis=1)    
    dataframe['Accession'] = dataframe.apply(extract_accession, axis=1)
    dataframe['GCF'] = dataframe['query-ID'].str.split("_").str[0:2].str.join('_')
    dataframe['ASM'] = dataframe['query-ID'].str.split("_").str[2]
    dataframe['Hash_1'] = dataframe['shared-hashes'].str.split("/").str[0]
    dataframe['Hash_2'] = dataframe['shared-hashes'].str.split("/").str[1]
    
    to_int = ['Hash_1', 'Hash_2']    
                
    for column in dataframe.columns:
        if column in to_int:
            dataframe[column] = dataframe[column].astype(int)
            
    dataframe['Hash_fr'] = dataframe['Hash_1']/ dataframe['Hash_2']
    
    return dataframe

In [118]:
df = import_mash_screen_to_pandas(file)

In [148]:
def extract_species_from_screen(screen_dataframe, identity_threshold=0.9):
    df_index = screen_dataframe[screen_dataframe.identity > identity_threshold]
    #max_hash = df_index['Hash_fr'].max()
    hash_values = df_th.Hash_fr.values.tolist()
    hash_values.sort(reverse=True)

    main_species = df_index['Species'][df_index['Hash_fr'] == hash_values[0]].values[0]
    
    species_report = "Main species: " + "<i>" + main_species + "</i>" + "<br />"
    
    print(len(hash_values))
    #<p style="padding-right: 5px;">My Text Here</p>.
    if len(hash_values) > 1:
        for index, hash_value in enumerate(hash_values):
            species_hashed = df_index['Species'][df_index['Hash_fr'] == hash_value].values[0]
            if index != 0:
                if hash_value > 0.6:
                    species_line = "Another high represented species found: " + "<i>" + species_hashed + "</i>" + "<br />"
                elif hash_value < 0.3:
                    species_line = "Another less represented species found: " + "<i>" + species_hashed + "</i>" + "<br />"
                else:
                    species_line = "Another mild represented species found: " + "<i>" + species_hashed + "</i>" + "<br />"
                    
                species_report = species_report + species_line
            print(index, hash_value)        
        
    
    return main_species, species_report
    

In [149]:
extract_species_from_screen(df)

2
0 0.998
1 0.137


('Mycobacterium tuberculosis',
 'Main species: <i>Mycobacterium tuberculosis</i><br />Another less represented species found: <i>Pseudomonas aeruginosa</i><br />')

In [129]:
df.head(2)

,identity,shared-hashes,median-multiplicity,p-value,query-ID,query-comment,Species,Accession,GCF,ASM,Hash_1,Hash_2,Hash_fr
0,0.719686,1/1000,1,0.003494,GCF_000008225.1_ASM822v1_genomic.fna.gz,"NC_007332.1 Mycoplasma hyopneumoniae 7448, com...",Mycoplasma hyopneumoniae,NC_007332.1,GCF_000008225.1,ASM822v1,1,1000,0.001
1,0.719686,1/1000,1,0.003494,GCF_000010425.1_ASM1042v1_genomic.fna.gz,NC_008618.1 Bifidobacterium adolescentis ATCC ...,Bifidobacterium adolescentis,NC_008618.1,GCF_000010425.1,ASM1042v1,1,1000,0.001


In [66]:
df_th = df[df.identity > .9]

In [67]:
df_th.head()

,identity,shared-hashes,median-multiplicity,p-value,query-ID,query-comment,Species,Accession,GCF,ASM,Hash_1,Hash_2,Hash_fr
453,0.999905,998/1000,52,0.0,GCF_001391675.1_6570_6_7_genomic.fna.gz,[164 seqs] NZ_CNKO01000001.1 Mycobacterium tub...,Mycobacterium tuberculosis,NZ_CNKO01000001.1,GCF_001391675.1,6570,998,1000,0.998
517,0.909686,137/1000,1,0.0,GCF_001451535.1_WH-SGI-V-07427_genomic.fna.gz,[130 seqs] NZ_LLPO01000001.1 Pseudomonas aerug...,Pseudomonas aeruginosa,NZ_LLPO01000001.1,GCF_001451535.1,WH-SGI-V-07427,137,1000,0.137
